# Reshaping

We've now seen some of the benefits of panel data, and how we can take advantage of pandas to manipulate it and get some insights. Sometimes though we'll need to reshape our data to work with it more easily.

In [3]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv("data/sp500_q1_2025.csv")

# Convert 'datadate' to datetime
df.DlyCalDt = pd.to_datetime(df.DlyCalDt)

# Data cleaning as we did previously
df.dropna(inplace=True)

print("Missing data after cleaning", df.isnull().sum().sum())

# Handling duplicates
print("Checking for duplicates, which we forgot to do previously!", df.duplicated().sum())

df.drop_duplicates(inplace=True)


Missing data after cleaning 0
Checking for duplicates, which we forgot to do previously! 2


## Pivot

`pivot` helps us reshape *long* panel data into a *wide* data frame. We can use it to have each stock in a separate column and dates in the rows. We can only have one column substituted in as the values of the data frame, so choose carefully.

In [ ]:
df.shape
pivot_df = df.pivot(index="DlyCalDt", columns="SecurityNm", values="DlyClose")
pivot_df.loc["2025-02"]

In [ ]:
pivot_df.isnull().sum().sum()
pivot_df["MCCORMICK & CO INC; COM V; CONS"] = pivot_df["MCCORMICK & CO INC; COM V; CONS"].ffill()
pivot_df.dropna(axis=1, inplace=True)
missing = pivot_df.isnull().sum()
missing[missing > 0]

We generally favour returns over close prices, as they give us a better picture of relative performance. Because our data frame is only holding close prices, it is straightforward to calculate returns.

In [ ]:
pivot_df.pct_change().mean()

The other really neat thing we can do with this kind of pivoted dataframe is visualise correlations with ease.

In [ ]:
pivot_df.pct_change().corr()

### Exercise: Trading Top Ten

Pivot our panel data, this time using trading volume `DlyVol` for values. Find the max trading volume for each stock and display the top 10.

In [ ]:
volume_pivot = df.pivot(index="DlyCalDt", columns="SecurityNm", values="DlyVol")
volume_pivot.max().nlargest(10)

## Resample

The other kind of reshaping we can do is called *resampling*, which we use to change the frequency of our data. When we resample, we are generally expected to do some aggregation (but we don't have to). Let's resample our pivoted data to get the mean closing price for each month.

In [ ]:
pivot_df.resample("ME").max()

pivot_df.resample("ME").asfreq().pct_change()


SecurityNm,3M CO; COM NONE; CONS,A E S CORP; COM NONE; CONS,A P A CORP; COM NONE; CONS,A T & T INC; COM NONE; CONS,ABBOTT LABORATORIES; COM NONE; CONS,ABBVIE INC; COM NONE; CONS,ACCENTURE PLC IRELAND; COM A; CONS,ADOBE INC; COM NONE; CONS,ADVANCED MICRO DEVICES INC; COM NONE; CONS,AFLAC INC; COM NONE; CONS,...,WILLIAMS COS; COM NONE; CONS,WILLIS TOWERS WATSON PUB LTD CO; COM NONE; CONS,WYNN RESORTS LTD; COM NONE; CONS,X C E L ENERGY INC; COM NONE; CONS,XYLEM INC; COM NONE; CONS,YUM BRANDS INC; COM NONE; CONS,ZEBRA TECHNOLOGIES CORP; COM A; CONS,ZIMMER BIOMET HOLDINGS INC; COM NONE; CONS,ZIONS BANCORPORATION N A; COM NONE; CONS,ZOETIS INC; COM A; CONS
DlyCalDt,,,,,,,,,,,,,,,,,,,,,
2025-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-28,0.019185,0.053636,-0.056088,0.155078,0.078793,0.136650,-0.094688,0.002537,-0.138767,0.019464,...,0.049612,0.030601,0.028440,0.072917,0.055224,0.198238,-0.196178,-0.047132,-0.066021,-0.021416
2025-03-31,-0.053249,0.071613,0.015459,0.031740,-0.038838,0.002344,-0.104620,-0.125479,0.028840,0.015712,...,0.027157,-0.005005,-0.065159,-0.018169,-0.087325,0.006331,-0.103126,0.084931,-0.077350,-0.015487


We can use `resample()` to help us calculate returns for different periods. When we calculated daily returns we took the last price on the day and the last price on the day before.

For other periods we apply the same thinking. For monthly returns, for example, we take the last price of the month and the last price of the month before. We'll need `last()` to make it work.

There are many possibilities for resampling, here are a few:

- **W** - Weekly
- **D** - Daily (calendar days)
- **QE** - Quarterly (quarter end)
- **YE** - Annually

### Exercise: Losing Days

Resample your *trading volume* pivot df to calendar days. Do you need to do some cleaning? What do you propose?

In [74]:
## YOUR CODE GOES HERE